## Data Processing 

In [1]:
#libraries for data cleaning
import pandas as pd
import numpy as np
import string
import nltk
#nltk.download("stopwords")
# from nltk.corpus import stopwords
# from nltk import regexp_tokenize
# from nltk import word_tokenize
# #nltk.download('punkt')
# from nltk import word_tokenize
import aspyr
import re
import os
import glob

In [2]:
sys.path.insert(3, "/data/home/xyuan/.local/lib/python3.6/site-packages")
import sys 
#sys.path.insert(0, "./library/python-ftfy")
import ftfy
import ftfy.bad_codecs 
#sys.path.insert(1, "./library/emoji")
import emoji
sys.path.insert(2, "./library/ipython-autotime")
import autotime
%load_ext autotime
import langid
from langid.langid import LanguageIdentifier, model 

In [3]:
#libraries for vectorization
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from scipy import sparse

time: 555 ms


In [4]:
#pd display options
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

time: 1.08 ms


In [5]:
#db connect
redshift = aspyr.db_connect.get_redshift_dw_connection()
aurora = aspyr.db_connect.get_aurora_dw_connection()

time: 1.01 s


In [139]:
#47855 rows don't have groups
df = pd.read_sql(
    """SELECT p.post_key,p.post_id,g.name, p.post_type, pt.title, pt.body, p.modr_status
    FROM ds.posts AS p
    LEFT JOIN ds.post_text AS pt ON p.post_key = pt.post_key
    LEFT JOIN bc.groups AS g ON p.group_id = g.group_id 
    WHERE EXTRACT(YEAR FROM p.t_created) IN (2017,2018,2019) AND p.post_type IN ('disc','jrnl')""", redshift)

time: 32.7 s


In [6]:
#New queried data 04/02/2019
df_new = pd.read_sql(
    """SELECT p.post_key,p.post_id,g.name, p.post_type, pt.title, pt.body, p.modr_status
    FROM ds.posts AS p
    LEFT JOIN ds.post_text AS pt ON p.post_key = pt.post_key
    LEFT JOIN bc.groups AS g ON p.group_id = g.group_id 
    WHERE EXTRACT(YEAR FROM p.t_created) IN (2017,2018,2019) AND p.post_type IN ('disc','jrnl')""", redshift)

time: 30.3 s


In [140]:
df.shape 

(182263, 7)

time: 2.47 ms


In [7]:
df_new.shape

(187352, 7)

time: 14.4 ms


In [21]:
#5089 new posts added, 4285 nonspam, 804 spams
new_posts = df_new[(df_new.post_key.isin(df_nonspam.post_key) == False) &(df_new.post_key.isin(df_spam.post_key) == False)]
new_nonspam = new_posts[new_posts.modr_status == 2] 
new_spam = new_posts[new_posts.modr_status == 3] 
#new_posts.to_csv("new_posts0402.csv", index = False)

time: 184 ms


### Data Overview
> Post created in the past 3 years:
- Years: 2017,2018,2019
- Post Type: disc, jrnl
- 7 Columns: post_key, post_id, group_name, post_type, title, body, modr_status
- Num of Rows: 182263

### Exporting Data
> Brute-force solution to memory error:
- Dump data into 10 batches *(20000 rows/batch)*
- Export data as *.csv*

In [149]:
#create data batches  
def slicer(df, n):
    j = 1
    for i in range(0, df.shape[0],n):
        df_output = df[i:i+n]
        df_output.to_csv("/data/home/xyuan/ml_project/data_batches/post17_19_{}.csv".format(j) , index=False)
        j = j+1

time: 1.54 ms


In [150]:
slicer(df, 20000) 

time: 4.16 s


In [158]:
del df

time: 678 µs


### Text Cleansing
>**Defined function: *post_fixer()***
- Replace URL (e.g. "http://www.shaperich.com/slimfit-180-4-for-life/" or "www.thyca.org/sg/") to plain text ('urlink')
- Fix unicode errors (e.g.  â€™ ('), â€, â€¢, â€œ）
- Remove Emojis (e.g. ðŸ¾ðŸŽ‰ðŸŽˆðŸŽ‰ðŸŽˆðŸ¾ = 🍾🎉🎈🎉🎈🍾) 
- Remove pure numbers
- Replace useless characters (e.g. new line characters)

### Tokenizing & Filtering Stopwords 
>**Defined function: *tokenizer()***
- Regular-Express Tokenizer (e.g. can't/ca n't) for better flexibility
- Enriched stopwords list (nltk stopwords + punctuations + manually-captured stopwords)

### Lemmitization Yes/No?
>No, might have a deleterious effect on IR but could have a try later

In [8]:
#bulk reading into a dictionary of dataframes     
def loaddata (path) :
    d = {}
    for file in os.listdir(path):
        if file.endswith(".csv"):
            d[file[:-4]] = pd.read_csv(path+'/'+file, encoding = 'utf-8')
            df = pd.concat(d, axis = 0)
            df.index = df.index.droplevel(0)
    df_nonspam = df[df.modr_status == 2]
    df_spam = df[df.modr_status == 3]
    return df_nonspam, df_spam

time: 2.16 ms


In [9]:
df_nonspam, df_spam = loaddata("./data_batches") #135836 rows, 46427 rows

time: 2.92 s


In [50]:
def langdetect(dataframe, text_column):
    result_list = []
    identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
    for i in range(0,len(dataframe[text_column].values)):
        dataframe[text_column].values[i] = dataframe[text_column].values[i].encode('sloppy-windows-1252').decode('utf-8')
        text = dataframe[text_column].values[i]
        result_list.append(identifier.classify(text.lower()))
    lan_list = np.array([w for w,c in result_list]).reshape((len(result_list),1))
    conf_list = np.array([c for w,c in result_list]).reshape((len(result_list),1))
    dataframe['lang'] = lan_list
    dataframe['confidence'] = conf_list
    return dataframe

time: 3.07 ms


In [51]:
test_df = langdetect(df_nonspam, "body") #135836 non-spam  
all_en = test_df[test_df.lang == "en"] #134997 non-spam & en, 839 non-soam & non-en

time: 6min 44s


In [2]:
#v0-functions
def data_cleanser(data):
    
    def post_fixer(post):
        post = re.sub('(http[s]?(\:\/\/)|(www\.))(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','urllink', post)
        post = post.encode('sloppy-windows-1252').decode('utf-8')
        filtered  = [w for w in re.split(r'\s+|[,;.!-%]\s*', post) if not any(i in w for i in emoji.UNICODE_EMOJI)]
        #remove pure numbers
        non_num = [w for w in filtered if not w.isdigit()]
        return " ".join(filter(None,non_num)) 

    def tokenizer(raw_string):
        stop = stopwords.words('english') + list(string.punctuation) + ["can't","i'm","i've","i'd"] #add urllink?
        if type(raw_string) is str:
            raw_string_lower = raw_string.lower()
            return [i for i in regexp_tokenize(raw_string_lower, "[\w']+") if i not in stop] 
    
    for i in range(0,len(list(d.keys()))):
        data = d[list(d.keys())[i]]
        data['cleaned'] = data['body'].apply(post_fixer)
        data['token'] = data['cleaned'].apply(tokenizer)
        data.to_pickle("./data_batches/post17_19_{}.pkl".format(i+1))

In [584]:
#v1-pipeline
import spacy
from spacy.attrs import LOWER, POS, ENT_TYPE, IS_ALPHA, TAG
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

#remove tokens matched in spaCy stopwords(pure lowercased stopwords without punctuations or shorthand expressions)
def stpwd_remover(doc):
    stopwords = spacy.lang.en.stop_words.STOP_WORDS
    def keep_token(token):
        if (token.norm_ not in stopwords) & (token.text not in stopwords):
            return token
    stp_removed = list(filter(keep_token,doc))
    result = ' '.join(token.string for token in stp_removed)
    return tokenizer(result) 
    
#remove tokens resolved as emoji    
def emoji_remover(doc):
    def keep_token(token):
        if not any(j in token.text for j in emoji.UNICODE_EMOJI):
            return token
    emoji_removed = list(filter(keep_token,doc))
    result = ' '.join(token.string for token in emoji_removed) 
    return tokenizer(result) 
            
#tagging urls and remove tokens have the tag as 'URL'    
def urleml_remover(doc):
    def keep_token(token):
        return token.tag_ not in {"URL","EMAIL"}
    for i, token in enumerate(doc):
        if token.like_url:
            token.tag_ = "URL"
        elif token.like_email:
            token.tag_ = "EMAIL"
    urleml_removed = list(filter(keep_token,doc))
    result = ' '.join(token.string for token in urleml_removed)
    return tokenizer(result)

#remove all tokens like:digit-numbers, whitespaces, punctutations,
def super_remover(doc):
    def keep_token(token):
        return token.pos_ not in {"FLAG"}
    for i, token in enumerate(doc):
        if token.like_num | token.is_punct | token.is_space:
            token.tag_ = "FLAG"
    super_removed = list(filter(keep_token,doc))
    result = ''.join(token.string for token in super_removed)
    return tokenizer(result)

#def remove tokens on_match

nlp = spacy.load('en')
nlp.add_pipe(stpwd_remover, name='stpwd_remover', first=False)
nlp.add_pipe(urleml_remover, name='urleml_remover', first=False)
nlp.add_pipe(emoji_remover, name='emoji_remover', first=False)
nlp.add_pipe(super_remover, name='super_remover', first=False)
tokenizer = English().Defaults.create_tokenizer(nlp)
print(nlp.pipe_names)

['tagger', 'parser', 'ner', 'stpwd_remover', 'urleml_remover', 'emoji_remover', 'super_remover']
time: 592 ms


In [9]:
#v2-pipeline 
import spacy
from spacy.attrs import LOWER, POS, ENT_TYPE, IS_ALPHA, TAG
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

#remove tokens resolved as emoji    
def emoji_remover(doc):
    def keep_token(token):
        if not any(j in token.text for j in emoji.UNICODE_EMOJI):
            return token
    emoji_removed = list(filter(keep_token,doc))
    result = ' '.join(token.string for token in emoji_removed) 
    return tokenizer(result)

def noun_selector(doc):
    def keep_token(token):
        if (token.pos_ in ('NOUN', 'PROPN')) & (token.is_alpha) & (token.is_stop == False):
            return token
    removed_result = list(filter(keep_token,doc))
    result = ''.join(token.string for token in removed_result)
    return tokenizer(result)

# def remove_tokens_onmatch(doc):
#     def keep_token(token):
#         if (token.pos_ not in ('PUNCT','NUM','SYM','SPACE')) & (token.is_alpha) & (token.is_stop == False):
#             return token
#     removed_result = list(filter(keep_token,doc))
#     result = ''.join(token.string for token in removed_result)
#     return tokenizer(result)

nlp = spacy.load('en')
nlp.add_pipe(emoji_remover, name='emoji_remover', first=True)
# nlp.add_pipe(remove_tokens_onmatch, name='pos_remover', first=False)
nlp.add_pipe(noun_selector, name='noun_selector', first= False)
tokenizer = English().Defaults.create_tokenizer(nlp)
print(nlp.pipe_names)

['emoji_remover', 'tagger', 'parser', 'ner', 'noun_selector']
time: 1.5 s


In [ ]:
#nlp = spacy.load('en')
text = all_en.body.values.tolist()
cleaned_text = list(map(lambda x : nlp(u"{}".format(x.lower())), text))

In [24]:
cleaned_df = pd.read_csv("cleaning_test_output.csv", encoding = "utf-8")

time: 2.63 s


/opt/default/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [40]:
cleaned_df.post_key.count()

134997

time: 9.35 ms


In [45]:
cleaned_df.to_json("cleaning_test_output.json", orient = 'records', lines = True)

time: 2.19 s


In [42]:
cleaned_df.loc[cleaned_df.cleaned_text.isnull() == True]

,post_key,post_id,name,post_type,title,body,modr_status,lang,confidence,cleaned_text
37,disc.749462,749462,Neurofibromatosis Network,disc,Experimental drug shows promise for disfiguring tumor condition,https://www.cancer.gov/news-events/pres-releases/2016/selumetinib-shrinks-pediatric-tumors\r\n\r\nhttp://www.nejm.org/doi/full/10.1056/NEJMoa1605943?query=featured_home#results\r\n\r\nhttp://www.upi.com/Health_News/2016/12/29/Experimental-drug-shows-promise-for-disfiguring-tumor-condition/3811483045629/,2,en,0.998655,NaN
53,disc.749584,749584,American Brain Tumor Association,disc,For consideration,https://youtu.be/8uY3bfU3vG4,2,en,0.169462,NaN
342,disc.751577,751577,Ehlers-Danlos Syndromes & Related Disorders,disc,Local Anesthesia Resistance story for EDS patients,http://www.bbc.com/future/story/20170106-the-people-who-cant-go-numb-at-the-dentists,2,en,0.962826,NaN
552,disc.753061,753061,American Brain Tumor Association,disc,New discovery research for brain cancer,https://www.sciencedaily.com/releases/2017/01/170117084050.htm,2,en,0.999853,NaN
753,disc.754436,754436,American Brain Tumor Association,disc,Challenges in Immunotherapy Presented by the GBM Microenvironment,https://www.hindawi.com/journals/jir/2011/732413/,2,en,0.738883,NaN
1015,disc.756332,756332,American Liver Foundation,disc,How can you tell if you are having an HE moment,I don't think I've had one yet but would like to know if you can tell if one is coming on or not?,2,en,1.000000,NaN
1452,disc.749608,749608,Ehlers-Danlos Syndromes & Related Disorders,disc,Our Need for Various Forms of Vit. K - and AFFECTS on GENETIC EXPRESSION,http://articles.mercola.com/sites/articles/archive/2017/01/02/what-are-the-benefits-of-vitamin-k2.aspx?utm_source=dnl&utm_medium=email&utm_content=art2&utm_campaign=20170102Z1&et_cid=DM130051&et_rid=1824517616,2,en,0.810548,NaN
1528,disc.750171,750171,American Brain Tumor Association,disc,Pre Clinical Immunotherapy + gene therapy shows promise in GBM,https://www.sciencedaily.com/releases/2017/01/170104143613.htm,2,en,0.999853,NaN
1898,disc.749951,749951,Ehlers-Danlos Syndromes & Related Disorders,disc,pharmacogenomics of pain management,A very interesting read: \r\nhttps://www.dovepress.com/the-pharmacogenomics-of-pain-management-prospects-for-personalized-med-peer-reviewed-fulltext-article-JPR,2,en,0.969902,NaN
1935,disc.752048,752048,Ehlers-Danlos Syndromes & Related Disorders,disc,Just interesting,This amazes me.\r\n\r\nhttp://www.smithsonianmag.com/smart-news/how-childrens-toy-may-help-fight-malaria-180961779/,2,en,0.999551,NaN


time: 59.6 ms


### Paragraph Embedding
>**Defined functions: *d2c_vectorizer()*, *tfidf_vectorizer()***
- Doc2Vec: using [pre-trained doc2vec model](https://github.com/jhlau/doc2vec#pre-trained-doc2vec-models) 
- TFIDF

In [20]:
#bulk reading into a dictionary of dataframes
os.chdir("./data_batches")
d = {}
for filename in glob.glob('*.pkl'):
        d[filename[:-4]] = pd.read_pickle(filename)

time: 5.07 s


In [21]:
#doc2vec
def d2c_vectorizer(dictionary):
    vec_list = []
    model = gensim.models.Doc2Vec.load('./d2c_model/doc2vec.bin') 
    for j in range(0, len(list(dictionary.keys()))):
        data = dictionary[list(dictionary.keys())[j]]
        for i in range(0, len(data['cleaned'])):
            doc_vectors = model.infer_vector(data['cleaned'][i])
            vec_list.append(doc_vectors)
    return np.asarray(vec_list)

#tfidf 
def tfidf_vectorizer(dictionary):
    df = pd.concat(dictionary)
    df.index = df.index.droplevel(0)
    vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'))
    matrix = vectorizer.fit_transform(df['cleaned'])
    return matrix

time: 3.14 ms


In [6]:
vec_all = d2c_vectorizer(d)

/opt/default/lib/python3.6/site-packages/gensim/models/doc2vec.py:531: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/opt/default/lib/python3.6/site-packages/gensim/models/doc2vec.py:535: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


time: 55min 26s


In [7]:
vec_all.shape

(176653, 300)

time: 8.62 ms


In [ ]:
pd.DataFrame(vec_all).to_json('./data_vec/post17_19_vec.json', orient = 'values')

In [24]:
matrix = tfidf_vectorizer(d)

time: 19.9 s


In [25]:
sparse.save_npz("./data_vec/tfidf_matrix.npz", matrix)

time: 9.12 s
